In [79]:
import os
import json

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# obtain skeletons + label

SKELETON_PATH = r"C:\Users\Ryan\VSCode Projects\CV_gesture_detection\skeletons_tensor.pt" # get path (put your own path here if you want to run)
META_PATH = r"C:\Users\Ryan\VSCode Projects\CV_gesture_detection\skeleton_annots.json"# get path
skeletons = torch.load(SKELETON_PATH)  


with open(META_PATH, "r") as f:
    metadata = json.load(f)

labels = torch.tensor(
    [sample["label_id"] for sample in metadata["samples"]],
    dtype=torch.long,
)

#filter out first 100 datapoints since they aren't labelled well
skeletons = skeletons[100:]
labels = labels[100:]

# sanity check stuff
print("Labels shape:", labels.shape)
print("Skeletons shape:", skeletons.shape)
print("Num classes:", len(torch.unique(labels)))

Labels shape: torch.Size([3781])
Skeletons shape: torch.Size([3781, 210, 21, 3])
Num classes: 14


In [81]:
# training stuff 
num_samples = skeletons.shape[0]
indices = torch.randperm(num_samples)
split = int(0.8 * num_samples) #80-20 split
train_idx = indices[:split]
test_idx   = indices[split:]

train_skeletons = skeletons[train_idx]
train_labels    = labels[train_idx]

test_skeletons = skeletons[test_idx]
test_labels    = labels[test_idx]

train_ds = TensorDataset(train_skeletons, train_labels)
test_ds   = TensorDataset(test_skeletons, test_labels)

batch_size = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader   = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

print("Train samples:", len(train_ds))
print("Test samples:", len(test_ds))

Train samples: 3024
Test samples: 757


In [ ]:
T, J, C = skeletons.shape[1:]  
input_dim = T * J * C
num_classes = len(torch.unique(labels)) + 1

print(num_classes)
class BasicGLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 420),
            nn.ReLU(),
            nn.Linear(420, 67),
            nn.ReLU(),
            nn.Linear(67, num_classes),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1) 
        return self.net(x)

device = torch.device("cpu")
model = BasicGLP(input_dim, num_classes).to(device)
print(model)

15
FlattenedCheeks(
  (net): Sequential(
    (0): Linear(in_features=13230, out_features=420, bias=True)
    (1): ReLU()
    (2): Linear(in_features=420, out_features=67, bias=True)
    (3): ReLU()
    (4): Linear(in_features=67, out_features=15, bias=True)
  )
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# training loop
def run_epoch(loader, model, criterion, optimizer=None):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0.0
    correct = 0
    total = 0

    for x_batch, y_batch in loader:
        x_batch = x_batch.to(device, dtype=torch.float32)
        y_batch = y_batch.to(device)

        if optimizer is not None:
            optimizer.zero_grad()

        logits = model(x_batch)           # forward pass
        loss = criterion(logits, y_batch) # compute loss

        if optimizer is not None:
            loss.backward()               # backprop
            optimizer.step()              # update weights

        total_loss += loss.item() * x_batch.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    avg_loss = total_loss / total
    acc = correct / total
    return avg_loss, acc

num_epochs = 15

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = run_epoch(train_loader, model, criterion, optimizer)
    test_loss, test_acc = run_epoch(test_loader, model, criterion, optimizer=None)

    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f}, train_acc={train_acc:.3f} | "
        f"test_loss={test_loss:.4f}, test_acc={test_acc:.3f}"
    )

Epoch 01 | train_loss=2.1957, train_acc=0.328 | test_loss=2.0058, test_acc=0.362
Epoch 02 | train_loss=1.9048, train_acc=0.424 | test_loss=1.7246, test_acc=0.501
Epoch 03 | train_loss=1.6239, train_acc=0.496 | test_loss=1.5656, test_acc=0.526
Epoch 04 | train_loss=1.4557, train_acc=0.529 | test_loss=1.3095, test_acc=0.568
Epoch 05 | train_loss=1.2591, train_acc=0.593 | test_loss=1.1989, test_acc=0.625
Epoch 06 | train_loss=1.1519, train_acc=0.633 | test_loss=1.1907, test_acc=0.618
Epoch 07 | train_loss=1.0422, train_acc=0.667 | test_loss=1.0044, test_acc=0.653
Epoch 08 | train_loss=1.0123, train_acc=0.667 | test_loss=1.0007, test_acc=0.668
Epoch 09 | train_loss=0.9645, train_acc=0.683 | test_loss=1.1512, test_acc=0.613
Epoch 10 | train_loss=0.8974, train_acc=0.701 | test_loss=1.1934, test_acc=0.590
Epoch 11 | train_loss=0.8787, train_acc=0.704 | test_loss=0.9667, test_acc=0.672
Epoch 12 | train_loss=0.8504, train_acc=0.718 | test_loss=1.0550, test_acc=0.657
Epoch 13 | train_loss=0.8640